## Autofocus multi-label prediction

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision import *

## Prepare the CSV

In [ ]:
import pandas as pd

In [ ]:
DATA_DIR = Path() / '..' / '..' / 'data' / 'lpz_2016_2017' / 'processed'
CSV_PATH_IN = DATA_DIR / 'labels.csv'

In [ ]:
autofocus_df = pd.read_csv(CSV_PATH_IN)

In [ ]:
autofocus_df.head()

In [ ]:
autofocus_df.loc[:, "location"] = autofocus_df.loc[:, "filename"].apply(
    lambda fn: fn.split("-")[2]
)

In [ ]:
autofocus_df = autofocus_df.loc[autofocus_df.loc[:, 'label'].notna(), ['filename', 'label', 'location']]

In [ ]:
autofocus_df.head()

In [ ]:
labelmap = {item:item for item in autofocus_df.loc[:, 'label'].unique()}

In [ ]:
labelmap['fox squirrel'] = 'squirrel'
labelmap['e. cottontail'] = 'rabbit'
labelmap["squirrel (can'\nsquirrel (can't ID)"] = 'squirrel'
labelmap["gray fox"] = 'fox'
labelmap["w. t. deer"] = 'deer'
labelmap['melanistic grey squirrel'] = 'squirrel'
labelmap['flying squirrel'] = 'squirrel'
labelmap['gray squirrel'] = 'squirrel'
labelmap['v. opossum'] = 'opossum'
labelmap['striped skunk'] = 'skunk'
labelmap['lawn mower'] = 'human'
labelmap['Mower'] = 'human'
labelmap['red fox'] = 'fox'

In [ ]:
autofocus_df.loc[:, 'label'] = autofocus_df.loc[:, 'label'].map(labelmap)

In [ ]:
autofocus_df.loc[:, 'label'].unique()

In [ ]:
autofocus_df = autofocus_df.groupby('filename').agg({'label': lambda x: ';'.join(x), 'location': min})

In [ ]:
autofocus_df = autofocus_df.reset_index()

In [ ]:
valid_locations = random.sample(autofocus_df.loc[:, 'location'].unique().tolist(), 10)

In [ ]:
autofocus_df.loc[:, 'valid'] = autofocus_df.loc[:, 'location'].isin(valid_locations)

In [ ]:
autofocus_df.to_csv('../../data/lpz_2016_2017/processed/multilabel.csv', index=False)

## Create the DataBunch

In [ ]:
src = (ImageList.from_csv(DATA_DIR, 'multilabel.csv', folder='images', suffix='')
       .split_by_rand_pct(0.1)
       .label_from_df(label_delim=';')
      )

In [ ]:
tfms = get_transforms()

In [ ]:
data = (src.transform(tfms, size=224)
        .databunch().normalize(imagenet_stats))

In [ ]:
data.show_batch(rows=3)

## Multiclassification

In [ ]:
arch = models.resnet50

In [ ]:
acc_02 = partial(accuracy_thresh, thresh=0.2)
f_score = partial(fbeta, thresh=0.2)
learn = create_cnn(data, arch, metrics=[acc_02, f_score])

We use the LR Finder to pick a good learning rate.

In [ ]:
# learn.lr_find()

In [ ]:
# learn.recorder.plot()

Then we can fit the head of our network.

In [ ]:
lr = 3e-2

In [ ]:
# learn.fit_one_cycle(2, slice(lr))

In [ ]:
# learn.save('stage-1-rn50')

...And fine-tune the whole model:

In [ ]:
# learn.unfreeze()

In [ ]:
# learn.load('stage-2-rn34')

In [ ]:
# learn.lr_find()
# learn.recorder.plot()

In [ ]:
# learn.fit_one_cycle(1, slice(1e-5, lr/5))

In [ ]:
# learn.save('stage-2-rn50')

In [ ]:
# learn.get_preds()

In [ ]:
# preds = _

In [ ]:
# human_probs = preds[0][:, 8]

In [ ]:
# human_pred = human_probs > .5

In [ ]:
# is_human = np.array([int(item[0] == 8) for item in learn.data.valid_ds.y.items])

In [ ]:
# from sklearn.metrics import accuracy_score, confusion_matrix, auc, roc_curve

In [ ]:
# accuracy_score(human_pred, is_human)

In [ ]:
# mat = confusion_matrix(is_human, human_pred)

In [ ]:
# mat

In [ ]:
# fpr, tpr, thresholds = roc_curve(is_human, human_probs, pos_label=1)

In [ ]:
# tpr

In [ ]:
# fig, ax = plt.subplots()
# plt.plot(fpr, tpr, color='darkorange')

In [ ]:
# from sklearn.metrics import fbeta_score

In [ ]:
# import numpy as np

# for thresh in np.linspace(0.1, .99, 99):
#     human_pred = human_probs > thresh
#     tp = sum(is_human & np.array(human_pred))
#     fp = sum(~is_human & np.array(human_pred))
#     fn = sum(is_human & ~np.array(human_pred))
#     precision = tp/(tp + fp)
#     recall = tp/(tp + fn)
#     fbeta = fbeta_score(y_true=is_human, y_pred=human_pred, beta=.1)
#     print(thresh, precision, recall, fbeta)

In [ ]:
!ls /home/ec2-user/image_drive/autofocus/data/lpz_2016_2017/processed/models

In [ ]:
data = (src.transform(tfms, size=512)
        .databunch(bs=32).normalize(imagenet_stats))

learn.data = data
learn.load('stage-2-256-rn50')

In [ ]:
preds = learn.get_preds()

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, auc, roc_curve

In [ ]:
thresh = .5

for labelnum in pd.Series(learn.data.valid_ds.y.items.sum()).value_counts().index:
    label = data.classes[labelnum]
    label_probs = preds[0][:, labelnum]
    has_label = np.array([int(item[0] == labelnum) for item in learn.data.valid_ds.y.items])
    
    label_pred = label_probs > thresh
    tp = sum(has_label & np.array(label_pred))
    tn = sum(~has_label & ~np.array(label_pred))
    fp = sum(~has_label & np.array(label_pred))
    fn = sum(has_label & ~np.array(label_pred))
    precision = tp/(tp + fp)
    recall = tp/(tp + fn)
    accuracy = (tp + tn)/(tp + tn + fp + fn)
    print(f'accuracy at {thresh}%: {accuracy}')
    print(f'precision at {thresh}%: {precision}')
    print(f'recall at {thresh}%: {recall}')

    fpr, tpr, thresholds = roc_curve(has_label, label_probs, pos_label=1)
    fig, ax = plt.subplots()
    ax.plot(fpr, tpr, color='darkorange')
    ax.set_title(label)
    plt.show()

In [ ]:
data = (src.transform(tfms, size=512)
        .databunch(bs=32).normalize(imagenet_stats))

learn.data = data
data.train_ds[0][0].shape

In [ ]:
learn.freeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
lr=1e-2/2

In [ ]:
learn.fit_one_cycle(5, slice(lr))

In [ ]:
learn.save('stage-1-512-rn50')

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, slice(1e-5, lr/5))

In [ ]:
learn.save('stage-2-256-rn50')

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.export()

In [ ]:
learn.load('stage-2-256-rn50')

In [ ]:
lr = 3e-2

In [ ]:
learn.fit_one_cycle(5, slice(1e-5, lr/5))

In [ ]:
learn.save('stage-3-512-rn50')

In [ ]:
learn.recorder.plot_losses()